# Handout 15

# **Concorrência e Threads**

## **1. Introdução às Threads**

Threads são unidades básicas de execução dentro de um processo. Diferentemente dos processos, que possuem espaços de memória independentes, múltiplas threads dentro do mesmo processo compartilham o mesmo espaço de memória, permitindo comunicação e compartilhamento de dados de forma eficiente.

**Vantagens das Threads:**
- **Desempenho:** Permitem a execução paralela de tarefas, aproveitando melhor os recursos de sistemas multicore.
- **Compartilhamento de Recursos:** Facilitam a comunicação entre tarefas, já que compartilham o mesmo espaço de memória.
- **Responsividade:** Melhoram a responsividade de aplicações, permitindo que tarefas de longa duração sejam executadas em segundo plano.

---

## **2. Criando e Gerenciando Threads com POSIX Threads (pthreads)**

A biblioteca POSIX Threads (pthreads) fornece uma API padrão para criação e gerenciamento de threads em sistemas Unix-like.

### **2.1. Criando uma Thread**

Para criar uma thread, utiliza-se a função `pthread_create`:

```c
#include <pthread.h>

int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine)(void *), void *arg);


- **thread:** Ponteiro para a variável que armazenará o identificador da thread criada.

- **attr:** Atributos da thread (pode ser NULL para atributos padrão).

- **start_routine:** Função que a thread executará.

- **arg:** Argumento passado para a função start_routine.

Exemplo:

In [ ]:
#include <pthread.h>
#include <stdio.h>

void *minha_thread(void *arg) {
    printf("Olá da thread!\n");
    return NULL;
}

int main() {
    pthread_t tid;
    pthread_create(&tid, NULL, minha_thread, NULL);
    pthread_join(tid, NULL);
    return 0;
}


**Explicação:**

- Define-se a função minha_thread que será executada pela thread.
- No main, cria-se a thread com pthread_create e espera-se sua conclusão com pthread_join

### **2.2. Esperando a Conclusão de uma Thread**

A função `pthread_join` é utilizada para aguardar a conclusão de uma thread específica:

```c
int pthread_join(pthread_t thread, void **retval);

```

- **thread:** Identificador da thread a ser aguardada
- **retval:** Ponteiro para o valor de retorno da thread (pode ser NULL se o valor não for necessário).

**Nota:** É importante utilizar pthread_join para evitar que a thread principal termine antes das threads criadas, o que poderia encerrar o processo antes da conclusão das threads.

## **3. Passando Argumentos para Threads**

Para passar argumentos para uma thread, utiliza-se o quarto parâmetro de pthread_create. Como esse parâmetro é do tipo void*, é comum passar ponteiros para estruturas que contêm os dados necessários.

Exemplo:


In [ ]:
#include <pthread.h>
#include <stdio.h>

struct thread_data {
    int id;
    char mensagem[100];
};

void *minha_thread(void *arg) {
    struct thread_data *data = (struct thread_data *)arg;
    printf("Thread %d diz: %s\n", data->id, data->mensagem);
    return NULL;
}

int main() {
    pthread_t tid;
    struct thread_data data = {1, "Olá da thread!"};
    pthread_create(&tid, NULL, minha_thread, &data);
    pthread_join(tid, NULL);
    return 0;
}


**Explicação:**

- Define-se uma estrutura thread_data para armazenar os dados a serem passados para a thread.
- No main, inicializa-se a estrutura e passa-se seu endereço como argumento para pthread_create.
- Na função minha_thread, converte-se o argumento recebido de volta para o tipo struct thread_data* e acessam-se os dados.

## **4. Retornando Valores de Threads**

Uma thread pode retornar um valor utilizando a função pthread_exit. O valor retornado pode ser recuperado pela thread que chamou pthread_join.

Exemplo:

In [ ]:
#include <pthread.h>
#include <stdio.h>

void *minha_thread(void *arg) {
    int *ret = malloc(sizeof(int));
    *ret = 42;
    pthread_exit((void *)ret);
}

int main() {
    pthread_t tid;
    int *resultado;
    pthread_create(&tid, NULL, minha_thread, NULL);
    pthread_join(tid, (void **)&resultado);
    printf("Resultado da thread: %d\n", *resultado);
    free(resultado);
    return 0;
}


**Explicação:**

- Na função minha_thread, aloca-se memória para um inteiro, atribui-se um valor e retorna-se o ponteiro utilizando pthread_exit.
- No main, após pthread_join, o ponteiro retornado é acessado para obter o resultado.
- É importante liberar a memória alocada após o uso para evitar vazamentos de memória.

## **5. Considerações sobre Concorrência**

Ao utilizar múltiplas threads que acessam recursos compartilhados, é essencial garantir que o acesso seja sincronizado para evitar condições de corrida (race conditions). Técnicas como mutexes e semáforos são utilizadas para controlar o acesso concorrente a recursos compartilhados.

Exemplo de Problema de Concorrência:

In [ ]:
#include <pthread.h>
#include <stdio.h>

int contador = 0;

void *incrementa(void *arg) {
    for (int i = 0; i < 1000000; i++) {
        contador++;
    }
    return NULL;
}

int main() {
    pthread_t t1, t2;
    pthread_create(&t1, NULL, incrementa, NULL);
    pthread_create(&t2, NULL, incrementa, NULL);
    pthread_join(t1, NULL);
    pthread_join(t2, NULL);
    printf("Contador final: %d\n", contador);
    return 0;
}


**Possível Resultado:**

- O valor esperado para contador seria 2.000.000, mas devido à falta de sincronização, o resultado pode ser menor, indicando uma condição de corrida.

**Solução:**

- Utilizar mecanismos de sincronização, como mutexes, para proteger o acesso à variável contador.